# Silver Planning Notebook

The goal of this notebook is pretty much just to inspect the bronze raw data, figure out what needs to cleaned whats useful whats redunant and come up with a silver schema that is suitbale for generating embeddings, and hierarchal clustering downstream.


In [6]:
#getting spark fired up
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, trim, to_date, to_timestamp,
    current_timestamp, length, from_json
)
from pyspark.sql.types import *
from delta import configure_spark_with_delta_pip

def build_spark():
    builder = (
        SparkSession.builder
        .appName("silver_transform")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.driver.memory", "6g")
        .config("spark.executor.memory", "6g")
        .config("spark.sql.shuffle.partitions", "16")
    )
    return configure_spark_with_delta_pip(builder).getOrCreate()

spark = build_spark()

from pathlib import Path

# silver notebook base dir
BASE_DIR = Path.cwd()  # /pipelines/silver

# bronze output (one level up + bronze folder)
BRONZE_DELTA = BASE_DIR.parent / "bronze" / "delta_news"

# silver output (current directory)
SILVER_DELTA = BASE_DIR / "delta_news_silver"

print("Bronze Delta path:", BRONZE_DELTA)
print("Silver Delta path:", SILVER_DELTA)

#loading bronze
df_bronze = spark.read.format("delta").load(str(BRONZE_DELTA))

print("\n BRONZE LOADED")
df_bronze.printSchema()

print("\n Bronze preview (5 rows):")
df_bronze.show(5, truncate=False)



Bronze Delta path: /home/david/School/CapStone/pipelines/bronze/delta_news
Silver Delta path: /home/david/School/CapStone/pipelines/silver/delta_news_silver

 BRONZE LOADED
root
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- extra_fields: string (nullable = true)
 |-- ingestion_ts: timestamp (nullable = true)
 |-- source_file: string (nullable = true)


 Bronze preview (5 rows):


+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Basic Inspection

In [3]:
#variety sample
import pyspark.sql.functions as F

df_bronze.orderBy(F.rand()).show(5, truncate=False)


+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
#contents of extra fields
raw_json = df_bronze.select("extra_fields").limit(1).collect()[0][0]
print(raw_json)


{"publication":"Reuters","author":"","url":"https://www.reuters.com/article/music-movies/bohemian-rhapsody-movie-makes-magic-for-queen-as-music-sales-soar-idUSL2N23X1QW","text_type":"headline+article","time_precision":"minute","dataset_source":"components.one:all-the-news-2 (via zip)","dataset":"all_the_news_2","source":"All the News (curation)","raw_type":"all_the_news_csv","tz_hint":"America/New_York"}


In [5]:
#article length distribution
from pyspark.sql.functions import length

df_bronze.select(length("text").alias("len")).describe().show()


+-------+-----------------+
|summary|              len|
+-------+-----------------+
|  count|          1972976|
|   mean|3074.756229168525|
| stddev| 3299.42615848981|
|    min|                0|
|    max|           188074|
+-------+-----------------+



In [6]:
#nulls:
from pyspark.sql.functions import col, sum

df_bronze.select([
    sum(col(c).isNull().cast("int")).alias(c + "_nulls")
    for c in df_bronze.columns
]).show()


+----------+----------+------------------+------------------+-----------------+
|date_nulls|text_nulls|extra_fields_nulls|ingestion_ts_nulls|source_file_nulls|
+----------+----------+------------------+------------------+-----------------+
|         0|         0|                 0|                 0|                0|
+----------+----------+------------------+------------------+-----------------+



In [14]:
#WAIT extra fields must be parsed and analysed seperately in terms of nulls

from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType

extra_schema = StructType([
    StructField("publication", StringType()),
    StructField("author", StringType()),
    StructField("url", StringType()),
    StructField("text_type", StringType()),
    StructField("time_precision", StringType()),
    StructField("dataset_source", StringType()),
    StructField("dataset", StringType()),
    StructField("source", StringType()),
    StructField("raw_type", StringType()),
    StructField("tz_hint", StringType()),
    StructField("date_raw", StringType()),
    StructField("date_trading", StringType()),
    StructField("anchor_policy", StringType())
])

df_exploded = df_bronze.withColumn("extra", from_json("extra_fields", extra_schema))
df_exploded.select("extra.*").show(5, truncate=False)
df_exploded.printSchema()


+-----------+----------------+---------------------------------------------------------------------------------------------------------+----------------+--------------+---------------------------------------+--------------+-----------------------+----------------+----------------+----------+--------------------+---------------------------------------------+
|publication|author          |url                                                                                                      |text_type       |time_precision|dataset_source                         |dataset       |source                 |raw_type        |tz_hint         |date_raw  |date_trading        |anchor_policy                                |
+-----------+----------------+---------------------------------------------------------------------------------------------------------+----------------+--------------+---------------------------------------+--------------+-----------------------+----------------+----------------

In [ ]:
from pyspark.sql.functions import countDistinct
#disticnt values in each column

distinct_counts = df_exploded.select([
    countDistinct(col(f"extra.{c}")).alias(c + "_distinct")
    for c in extra_schema.fieldNames()
])

distinct_counts.show(truncate=False)

#ill remove columns with only one distinct value as they add no information

+--------------------+---------------+------------+------------------+-----------------------+-----------------------+----------------+---------------+-----------------+----------------+-----------------+---------------------+----------------------+
|publication_distinct|author_distinct|url_distinct|text_type_distinct|time_precision_distinct|dataset_source_distinct|dataset_distinct|source_distinct|raw_type_distinct|tz_hint_distinct|date_raw_distinct|date_trading_distinct|anchor_policy_distinct|
+--------------------+---------------+------------+------------------+-----------------------+-----------------------+----------------+---------------+-----------------+----------------+-----------------+---------------------+----------------------+
|15                  |119847         |1972976     |1                 |2                      |1                      |1               |1              |1                |2               |1415             |979                  |1                     |


In [8]:
from pyspark.sql.functions import expr
#checking data types in extra fields

type_check = df_exploded.select([
    expr(f"typeof(extra.{c})").alias(c + "_type")
    for c in extra_schema.fieldNames()
]).limit(1)

type_check.show(truncate=False)


+----------------+-----------+--------+--------------+-------------------+-------------------+------------+-----------+-------------+------------+-------------+-----------------+------------------+
|publication_type|author_type|url_type|text_type_type|time_precision_type|dataset_source_type|dataset_type|source_type|raw_type_type|tz_hint_type|date_raw_type|date_trading_type|anchor_policy_type|
+----------------+-----------+--------+--------------+-------------------+-------------------+------------+-----------+-------------+------------+-------------+-----------------+------------------+
|string          |string     |string  |string        |string             |string             |string      |string     |string       |string      |string       |string           |string            |
+----------------+-----------+--------+--------------+-------------------+-------------------+------------+-----------+-------------+------------+-------------+-----------------+------------------+



In [ ]:
#summmary basically % missing
from pyspark.sql.functions import col, trim, length

def meaningful(c):
    return ((col(c).isNotNull()) & (trim(col(c)) != ""))

silver_check = df_exploded.select(
    meaningful("text").cast("int").alias("has_text"),
    meaningful("extra.date_raw").cast("int").alias("has_date_raw"),
    meaningful("extra.date_trading").cast("int").alias("has_date_trading"),
    meaningful("extra.publication").cast("int").alias("has_publication"),
    meaningful("extra.url").cast("int").alias("has_url")
)

silver_check.summary().show()


+-------+--------------------+------------------+------------------+---------------+-------+
|summary|            has_text|      has_date_raw|  has_date_trading|has_publication|has_url|
+-------+--------------------+------------------+------------------+---------------+-------+
|  count|             1972976|           1972976|           1972976|        1972976|1972976|
|   mean|  0.9999893561807138|0.4139954059248567|0.4139954059248567|            1.0|    1.0|
| stddev|0.003262470136270...|0.4925477974380908|0.4925477974380908|            0.0|    0.0|
|    min|                   0|                 0|                 0|              1|      1|
|    25%|                   1|                 0|                 0|              1|      1|
|    50%|                   1|                 0|                 0|              1|      1|
|    75%|                   1|                 1|                 1|              1|      1|
|    max|                   1|                 1|                 1|  

pretty much all the data has text so limited cleaning needed, dates and trading dates being misisng are annoying but theres also a top level date column with 100% of the data so not a massive concern

## Silver Schema — `delta_news_silver`

The Silver layer stores a clean, structured version of the Bronze news data.  
It flattens the `extra` struct, preserves useful metadata, and keeps the top-level `date`  
as the **single canonical publication date**, since Bronze already provides 100% coverage.

Silver is intentionally light-touch: minimal cleaning, no lossy transforms.

---

## 1. Columns Inherited Directly from Bronze (Flattened)

| Column         | Type      | Description                                                    |
|----------------|-----------|----------------------------------------------------------------|
| text           | string    | Full article text (trimmed; empty rows filtered).              |
| date           | date      | **Primary publication date with full coverage.**               |
| publication    | string    | From `extra.publication` (lowercased, trimmed).                |
| author         | string    | From `extra.author`.                                           |
| url            | string    | Canonical article URL.                                         |
| text_type      | string    | From `extra.text_type`.                                        |
| time_precision | string    | Granularity indicator (“day”, “minute”).                       |
| date_raw       | string    | Optional raw publication date.                                 |
| date_trading   | string    | Trading-aligned timestamp.                                     |
| tz_hint        | string    | Timezone hint.                                                 |
| dataset_source | string    | Dataset provenance.                                            |
| dataset        | string    | Dataset label.                                                 |
| source         | string    | Dataset source (curation pipeline).                            |
| raw_type       | string    | Dataset ingestion format.                                      |
| anchor_policy  | string    | Timestamp alignment policy.                                    |
| source_file    | string    | Originating Bronze Parquet file.                               |
| ingestion_ts   | timestamp | Bronze ingestion timestamp.                                    |

All metadata is kept — nothing dropped prematurely.

---

## 2. New Columns Created in Silver

| Column              | Type      | Description                                                |
|---------------------|-----------|------------------------------------------------------------|
| len_text            | integer   | Length of cleaned text for QA.                            |
| silver_ingestion_ts | timestamp | Timestamp of Silver transformation.                       |

---

## 3. Columns Dropped in Silver

| Dropped Column | Reason                         |
|----------------|--------------------------------|
| extra_fields   | Removed after flattening struct|

Everything else is retained.

---

## 4. Final Silver Table Schema

| Column               | Type      | Description                                               |
|----------------------|-----------|-----------------------------------------------------------|
| text                 | string    | Cleaned article text.                                     |
| date                 | date      | Canonical publication date (100% complete).               |
| publication          | string    | Normalised publication name.                              |
| author               | string    | Article author.                                           |
| url                  | string    | Canonical URL.                                            |
| text_type            | string    | Format identifier.                                        |
| time_precision       | string    | Timestamp granularity.                                    |
| date_raw             | string    | Optional raw publication date.                            |
| date_trading         | string    | Trading-aligned timestamp.                                |
| tz_hint              | string    | Timezone hint.                                            |
| dataset_source       | string    | Provenance info.                                          |
| dataset              | string    | Dataset label.                                            |
| source               | string    | Dataset source (curation).                                |
| raw_type             | string    | Dataset ingestion format.                                 |
| anchor_policy        | string    | Alignment policy from dataset.                            |
| len_text             | integer   | Character length of article text.                         |
| source_file          | string    | Original Bronze source.                                   |
| ingestion_ts         | timestamp | Bronze ingestion time.                                    |
| silver_ingestion_ts  | timestamp | Silver transformation time.                               |

---

# Approach to missing data

Approach to Missing Data

The Bronze layer contains a very small fraction of rows with missing core fields (primarily empty text or missing top-level date). These fields are essential for all downstream processing: text is required for embeddings and clustering, and date is required for daily aggregation and financial alignment.
Because the proportion of rows with missing core fields is extremely small, these observations are dropped during the Silver transformation without materially affecting coverage or representativeness.

In contrast, most other fields in the extra metadata block (such as author, tz_hint, dataset_source, or date_raw) are occasionally missing or contain empty strings. These fields are useful but not essential. They improve interpretability and allow for additional analysis but do not affect the core modelling pipeline. For this reason:

Missing core fields → dropped entirely

Missing metadata fields → retained as empty / null

This preserves the maximum amount of usable text data while maintaining clean guarantees for the fields required by the later Gold and embedding steps. The Silver layer therefore ensures strict completeness where it matters (text and date), and flexibility where metadata is optional.



steps-

implement above changes...

so py script that reads bronze. flattens extra fields, corrects for data types (mostly just dates), drops missing core fields, timestamps these changes and creates the len text col, write to silver delta table

bet


In [4]:
from pathlib import Path

# Path to Silver Delta output
SILVER_DELTA = Path("/home/david/School/CapStone/pipelines/silver/delta_news_silver")

# Load DataFrame
df_silver = spark.read.format("delta").load(str(SILVER_DELTA))

print("Silver table loaded ✓")

# Schema
df_silver.printSchema()

# Peek at 5 rows (truncate long text for readability)
df_silver.select(
    "date",
    "publication",
    "author",
    "url",
    "text_type",
    "time_precision",
    "date_raw",
    "date_trading",
    "dataset",
    "source",
    "raw_type",
    "tz_hint",
    "len_text",
    "source_file",
    "ingestion_ts",
    "silver_ingestion_ts"
).show(5, truncate=0)


Silver table loaded ✓
root
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- ingestion_ts: timestamp (nullable = true)
 |-- source_file: string (nullable = true)
 |-- publication: string (nullable = true)
 |-- author: string (nullable = true)
 |-- url: string (nullable = true)
 |-- text_type: string (nullable = true)
 |-- time_precision: string (nullable = true)
 |-- dataset_source: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- source: string (nullable = true)
 |-- raw_type: string (nullable = true)
 |-- tz_hint: string (nullable = true)
 |-- date_raw: string (nullable = true)
 |-- date_trading: string (nullable = true)
 |-- anchor_policy: string (nullable = true)
 |-- len_text: integer (nullable = true)
 |-- silver_ingestion_ts: timestamp (nullable = true)



+--------------------+-----------+-----------------------+----------------------------------------------------------------------------------------+----------------+--------------+----------+--------------------+--------------+-----------------------+----------------+----------------+--------+-----------------------------------------------------------------------------------+--------------------------+-------------------------+
|date                |publication|author                 |url                                                                                     |text_type       |time_precision|date_raw  |date_trading        |dataset       |source                 |raw_type        |tz_hint         |len_text|source_file                                                                        |ingestion_ts              |silver_ingestion_ts      |
+--------------------+-----------+-----------------------+--------------------------------------------------------------------------------

In [5]:
(df_silver
    .select("publication", "text")
    .orderBy("len_text", ascending=False)
    .limit(3)
    .show(truncate=False))


+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------